# Starbucks satisfaction survey. Hypothesis testing
Por: Francisco Jose Diaz - Data analyst de El Salvador

El presente notebook tiene como objetivo construir pruebas de hipotesis utilizando librerias de python. Los datos son de la encuesta de satisfaccion del consumidor de Starbucks Malasia 

Fuente de los datos: [Kaggle](https://www.kaggle.com/datasets/mahirahmzh/starbucks-customer-retention-malaysia-survey?select=Starbucks+satisfactory+survey+encode+cleaned.csv)

## Setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats.distributions as dist
from random import randrange

In [2]:
df = pd.read_csv("Starbucks satisfactory survey encode cleaned.csv")

## Explorando los datos 

In [3]:
df.head()

,Id,gender,age,status,income,visitNo,method,timeSpend,location,membershipCard,...,chooseRate,promoMethodApp,promoMethodSoc,promoMethodEmail,promoMethodDeal,promoMethodFriend,promoMethodDisplay,promoMethodBillboard,promoMethodOthers,loyal
0,1,1,1,0,0,3,0,1,0,0,...,3,1,1,1,1,1,1,1,1,0
1,2,1,1,0,0,3,2,0,1,0,...,2,1,1,1,1,1,1,1,1,0
2,3,0,1,2,0,2,0,1,2,0,...,3,1,1,1,1,1,1,1,1,0
3,4,1,1,0,0,3,2,0,2,1,...,3,1,1,1,1,1,1,1,1,1
4,5,0,1,0,0,2,2,1,1,1,...,3,1,1,1,1,1,1,1,1,0


## Prueba de hipotesis 
La prueba de hipotesis es una herramienta fundamental para determinar cual podria ser el valor de un parametro. Usaremos la encuesta de Starbucks para analizar casos de diferencias entre poblaciones y para comparar medias.

Para hacer mas entretenido este notebook vamos a modificar algunas variables. Se tiene en cuenta que este notebook es con fines didacticos para practicar estadistica inferencial.

Modificamos algunas variables codificadas a valores cuantitativos

In [4]:
df["incomeNumber"]=df.income.replace({0:randrange(1000,4999), 1:randrange(5000,9999), 2:randrange(10000,19999), 3:randrange(20000,29999), 3:randrange(30000,45000)})

In [5]:
df["ageNumber"]=df.age.replace({0:randrange(15,19), 1:randrange(20,29), 2:randrange(30,39), 3:randrange(40,60)})

Para el analisis de proporciones usaremos variables categoricas por lo tanto modificamos unas variables debido a que la encuesta se encuentra codificada.

In [6]:
df["genderx"]=df.gender.replace({0:"Male", 1:"Female"})
df["membershipCardx"]=df.membershipCard.replace({0:"Yes",1:"No"})
df["serviceRatex"]=df.serviceRate.replace({1:"Bad",2:"Bad",3:"Bad", 4:"Good", 5:"Good"})

## Prueba de hipotesis para una proporcion
Supongamos que en encuestas de Starbucks de otro pais se tiene que el 40% de los consumidores son del genero femenino. Por lo tanto se construye una prueba de hipotesis (two-sided) de que la proporcion de consumidores mujeres es del 40%.

**Population**: Consumidores de Starbucks  
**Parameter of Interest**: p  
**Null Hypothesis:** p = 0.40  
**Alternative Hypthosis:** p $\neq$ 0.40

Se desea saber si el 40% de los consumidores son mujeres

In [7]:
x = df.genderx.dropna() == "Female"  # segmentamos la poblacion de interes
p = x.mean() # obtenemos su media
p

0.5221238938053098

In [8]:
se = np.sqrt(.4 * (1 - .4)/ len(x)) 
se

0.04608572235855411

In [9]:
test_stat = (p - 0.4) / se
pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))
print("Valor test_stat", test_stat)
print("Con un p_valor de",
pvalue)

Valor test_stat 2.6499290356168617
Con un p_valor de 0.008050867934535846


**Insight**

Como resultado tenemos un pvalor menor a la significacia del 0.05 por lo que rechazamos la Hipotesis nula. Por lo tanto los consumidores del genero femenino no es un 40% de la poblacion. 

Realizando el mismo analisis pero con la libreria statsmodels:

In [10]:
n = 113
pnull = .40
phat = p
sm.stats.proportions_ztest(phat * n, n, pnull, alternative='two-sided', prop_var=0.4)

(2.6499290356168617, 0.008050867934535846)

## Prueba de hipotesis para dos proporciones 
Para realizar este analisis vamos a formular la siguiente pregunta:
¿es significante la diferencia entre la proporcion de consumidores masculinos y femeninos que reportan poseer tarjeta de membresia? 

**Populations**: Consumidores de Starbucks que tienen tarjeta de membresia  
**Parameter of Interest**: p1 - p2, where p1 = female and p2 = male  
**Null Hypothesis:** p1 - p2 = 0  
**Alternative Hypthosis:** p1 - p2 $\neq$ 0 

In [11]:
dx = df[["membershipCardx", "genderx"]].dropna()  # dropna drops cases where either variable is missing
pd.crosstab(dx.membershipCardx, dx.genderx)

genderx,Female,Male
membershipCardx,,
No,29,24
Yes,30,30


In [12]:
dz = dx.groupby(dx.genderx).agg({"membershipCardx": [lambda x: np.mean(x=="Yes"), np.size]})
dz.columns = ["Proportion", "N"] # The default column names are unclear, so we replace them here
print(dz) 

# The pooled rate of yes responses, and the standard error of the estimated difference of proportions
p_hat2 = (dx.membershipCardx == "Yes").mean()
va = p_hat2 * (1 - p_hat2)
se = np.sqrt(va * (1 / dz.N.Female  + 1 / dz.N.Male))

# Calculate the test statistic and its p-value
test_stat = (0.508475 - 0.555556) / se
pvalue = 2*dist.norm.cdf(-np.abs(test_stat))

print("\nEl test_stat es", (test_stat))
print("El pvalor para esta prueba es", (pvalue))

         Proportion   N
genderx                
Female     0.508475  59
Male       0.555556  54

El test_stat es -0.500949821465002
El pvalor para esta prueba es 0.6164064379282452


**Insight**

Se presenta un pvalor mayor al 0.05 por lo que se falla en rechazar la hipotesis nula. Por lo tanto no hay diferencia estadisticamente significativa entre la proporcion de consumidores masculinos y femeninos que reportan poseer tarjeta de membresia. 

Realizando el mismo analisis pero con la libreria statsmodels:

In [13]:
successes = np.array([30, 30])  # número de éxitos en cada grupo
totals = np.array([59, 54])  # tamaño de muestra en cada grupo

stat, p_value = sm.stats.proportions_ztest(successes, totals)

In [14]:
print(f"Con una prueba estadistica del {stat:.4f}")
print(f"y un valor p del {p_value:.3f}")
if p_value < 0.05:
    print("Hay evidencia suficiente para rechazar la hipotesis nula.")
else:
    print("No hay suficiente evidencia para rechazar la hipotesis nula.")


Con una prueba estadistica del -0.5009
y un valor p del 0.616
No hay suficiente evidencia para rechazar la hipotesis nula.


### Prueba de hipotesis para medias

#### Una Media

Para esta seccion buscaremos probar la pregunta: 
¿es el salario promedio de los consumidores de Starbucks superior a $5,000.00 dolares anuales?

**Population**: Todos los consumidores 
**Parameter of Interest**: $\mu$, salario promedio de los consumidores.
**Null Hypothesis:** $\mu$ = 5000
**Alternative Hypthosis:** $\mu$ > 5000

In [15]:
n = len(df)
mean = df["incomeNumber"].mean()
sd = df["incomeNumber"].std()
(n, mean, sd)

(113, 5627.1061946902655, 5903.412938962318)

In [16]:
test_stat, p_value = sm.stats.ztest(df["incomeNumber"], value = 5000, alternative = "larger")

**Insight**

In [17]:
print(f"Con una prueba estadistica del {test_stat:.4f}")
print(f"y un valor p del {p_value:.7f}")
if p_value < 0.05:
    print("Hay evidencia suficiente para rechazar la hipotesis nula.")
else:
    print("No hay suficiente evidencia para rechazar la hipotesis.") 

Con una prueba estadistica del 1.1292
y un valor p del 0.1294033
No hay suficiente evidencia para rechazar la hipotesis.


#### Diferencia en medias 

¿Considerando a los consumidores, las personas que dan serviceRate malos tienen una edad promedio diferente a quienes dan serviceRate buenos?

**Population**: Consumidores de Starbucks.  
**Parameter of Interest**: $\mu_1 - \mu_2$, Edad promedio. Donde
$\mu_1$ es edad promedio de quienes dan mala califacacion $\mu_2$ edad promedio de consumidores que dan calificacion positiva.

**Null Hypothesis:** $\mu_1 = \mu_2$  No hay diferencia 

**Alternative Hypthosis:** $\mu_1 \neq \mu_2$ Si hay diferencia

In [18]:
# Cruzamos variables para obtener su media, std y su size 
df.groupby("serviceRatex").agg({"ageNumber": [np.mean, np.std, np.size]})

ageNumber               
                   mean       std size
serviceRatex                          
Bad           25.525000  6.763353   40
Good          26.123288  6.105428   73

In [19]:
#Filtramos y creamos objetos
bad = df[df["serviceRatex"] == "Bad"]
good = df[df["serviceRatex"] == "Good"]

In [20]:
n1 = len(bad)
mu1 = bad["ageNumber"].mean()
sd1 = bad["ageNumber"].std()

In [21]:
n2 = len(good)
mu2 = good["ageNumber"].mean()
sd2 = good["ageNumber"].std()

In [22]:
# corremos la prueba con statsmodels 
test, p_value = sm.stats.ztest(bad["ageNumber"].dropna(), good["ageNumber"].dropna())

**Insight**

In [23]:
print(f"Con una prueba estadistica del {test:.4f}")
print(f"y un valor p del {p_value:.7f}")
if p_value < 0.05:
    print("Hay evidencia suficiente para rechazar la hipotesis nula.")
else:
    print("No hay suficiente evidencia para rechazar la hipotesis.") 

Con una prueba estadistica del -0.4794
y un valor p del 0.6316732
No hay suficiente evidencia para rechazar la hipotesis.
